## Data preparation for encoding/decoding models

For this example, we will use the data from Haxby et al., 2001., which are shared via OpenNeuro:

https://openneuro.org/datasets/ds000105/versions/3.0.0

The data are formatted according to the BIDS standard: https://bids-specification.readthedocs.io/en/stable/index.html

First, import required dependencies. You can install these using `pip install -r requirements.txt` from the main repo directory.

In [1]:
%load_ext autoreload
%autoreload 2

import os
import nilearn
from nilearn import datasets
from nilearn.maskers import NiftiMasker
from nilearn.maskers import MultiNiftiMapsMasker
import h5py
import numpy as np
import nibabel as nib
import datalad.api as dl
import pandas as pd
import tqdm
from utils import (
    get_subject_common_brain_mask,
    get_group_common_mask,
    get_subject_runs,
    get_layouts,
)
from pathlib import Path
from poldracklab.utils.run_shell_cmd import run_shell_cmd



#### Get the data using datalad

We will use a tool called [Datalad](https://www.datalad.org/) to obtain the data from openneuro.

We will download the raw data, as well as the processed data (using [fMRIPrep](https://fmriprep.org/en/stable/).  Note that downloading these derivative data can take quite a while depending on the speed of one's connection.

In [2]:
data_dir = '/Users/poldrack/data_unsynced/ds000105'
assert os.path.exists(data_dir), 'Data directory not found: %s' % data_dir
fmriprep_dir = os.path.join(data_dir, 'derivatives', 'fmriprep')

output_dir = os.path.join(data_dir, 'derivatives', 'cleaned')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)


# get the raw data
ds = dl.clone(
    path=data_dir,
    source='https://github.com/OpenNeuroDatasets/ds000105.git',
)
dl.get(dataset=data_dir, recursive=True)

get_fmriprep = False  # set to false after downloading fmriprep once

# get the preprocessed derivatives - this takes some time!
if get_fmriprep:
    dl.clone(
        path=fmriprep_dir,
        source='https://github.com/OpenNeuroDerivatives/ds000105-fmriprep.git',
    )
    dl.get(dataset=fmriprep_dir, recursive=True)

[INFO] Ensuring presence of Dataset(/Users/poldrack/data_unsynced/ds000105) to get /Users/poldrack/data_unsynced/ds000105 


### Query the dataset using PyBIDS

Because the dataset is organized using the BIDS standard, we can use the [PyBIDS](https://bids-standard.github.io/pybids/) tool to query the dataset and obtain useful metadata.


In [3]:
# load the dataset using pybids and get runs for each subject


layout, deriv_layout = get_layouts(data_dir, fmriprep_dir)

### create 3mm resolution version of the data

The data obtained from OpenNeuro were stored in MNI space at 2 mm resolution.  It's useful to work in MNI space in order to be able to align subjects, but the original resolution (3.5 x 3.75 x 3.75 mm) is much lower, and using the higher resolution images needlessly slows down analyses.  For this reason, we will resample the 2 mm data to 3 mm resolution, which is closer to the original resolution and will reduce processing times. 

In [4]:
boldfiles = deriv_layout.get(
    desc='preproc',
    space='MNI152NLin2009cAsym',
    suffix='bold',
    extension='nii.gz',
    res=2,
    return_type='file',
)
print(f'found {len(boldfiles)} bold files')

target_affine = np.diag((3, 3, 3))

for boldfile in tqdm.tqdm(boldfiles, desc='Processing items'):
    output = boldfile.replace('res-2', 'res-3')
    assert output != boldfile
    if not os.path.exists(output):
        img = nilearn.image.resample_img(
            boldfile,
            target_affine=target_affine,
            copy_header=True,
            force_resample=True,
        )
        img.to_filename(output)
    # create boldref as well
    boldref = boldfile.replace('desc-preproc_bold.nii.gz', 'boldref.nii.gz')
    output_boldref = boldref.replace('res-2', 'res-3')
    assert output_boldref != boldref
    if not os.path.exists(output_boldref):
        img = nilearn.image.resample_img(
            boldref,
            target_affine=target_affine,
            copy_header=True,
            force_resample=True,
        )
        img.to_filename(output_boldref)
    # create mask
    maskfile = boldfile.replace(
        'desc-preproc_bold.nii.gz', 'desc-brain_mask.nii.gz'
    )
    output_maskfile = maskfile.replace('res-2', 'res-3')
    assert output_maskfile != maskfile
    if not os.path.exists(maskfile):
        img = nilearn.image.resample_img(
            maskfile,
            target_affine=target_affine,
            interpolation='nearest',
            copy_header=True,
            force_resample=True,
        )
        img.to_filename(output_maskfile)

found 71 bold files


Processing items: 100%|██████████| 71/71 [00:00<00:00, 35835.81it/s]


### Confound regression

Use the outputs from fMRIPrep to generate a denoised version of the data.



In [5]:
def run_confound_regression(
    layout, deriv_layout, data_dir, res=3, overwrite=False
):
    cleaned_images = {}
    print(f'Running confound regression for res={res}')

    subjects = [int(sub) for sub in layout.get_subjects()]
    for subject in subjects:
        cleaned_images[subject] = {}
        runs = get_subject_runs(subject, data_dir)
        print(f'Subject {subject} has {len(runs)} runs')
        mask_img = get_subject_common_brain_mask(subject, data_dir, res=res)
        # mask_img = resample_img(mask_img, target_affine=np.diag((3, 3, 3)),
        #                        interpolation='nearest',
        #                        copy_header=True, force_resample=True)
        t_r = None
        for run in runs:
            preproc_file = deriv_layout.get(
                subject=subject,
                run=run,
                res=res,
                desc='preproc',
                space='MNI152NLin2009cAsym',
                suffix='bold',
                extension='nii.gz',
                return_type='file',
            )
            cleaned_img_file = preproc_file[0].replace('preproc', 'cleaned')
            if t_r is None:
                t_r = deriv_layout.get_metadata(preproc_file[0])[
                    'RepetitionTime'
                ]
            else:
                assert (
                    t_r
                    == deriv_layout.get_metadata(preproc_file[0])[
                        'RepetitionTime'
                    ]
                )
            assert t_r is not None
            if os.path.exists(cleaned_img_file) and not overwrite:
                # print(f"Using existing cleaned file for subject {subject} run {run}")
                cleaned_img = nib.load(cleaned_img_file)
                cleaned_images[subject][run] = (cleaned_img_file, cleaned_img)
                continue
            preproc_img = nib.load(preproc_file[0])
            assert (
                len(preproc_file) == 1
            ), f'Found {len(preproc_file)} preproc files for subject {subject} run {run}'
            confound_file = deriv_layout.get(
                subject=subject,
                run=run,
                desc='confounds',
                suffix='timeseries',
                extension='tsv',
                return_type='file',
            )
            assert (
                len(confound_file) == 1
            ), f'Found {len(confound_file)} confound files for subject {subject} run {run}'
            confounds = pd.read_csv(confound_file[0], sep='\t').bfill()
            # need to include cosine with acompcor
            confound_prefixes = ['trans_', 'rot_', 'a_comp_cor_', 'cosine']
            confound_cols = [
                c
                for c in list(confounds.columns)
                if any([c.startswith(p) for p in confound_prefixes])
            ]
            confounds_selected = confounds[confound_cols]
            cleaned_img = nilearn.image.clean_img(
                preproc_img,
                confounds=confounds_selected,
                t_r=t_r,
                mask_img=mask_img,
            )
            assert cleaned_img_file != preproc_file[0]
            cleaned_img.to_filename(os.path.join(cleaned_img_file))
            cleaned_images[subject][run] = (cleaned_img_file, cleaned_img)
    return cleaned_images, t_r


# refresh the layouts to detect the new res-3 files
layout, deriv_layout = get_layouts(data_dir, fmriprep_dir)

cleaned_images, t_r = run_confound_regression(
    layout, deriv_layout, data_dir, overwrite=False, res=3
)

# run it for the 2 mm resolution data as well
cleaned_images, t_r = run_confound_regression(
    layout, deriv_layout, data_dir, overwrite=False, res=2
)

Running confound regression for res=3
Subject 1 has 12 runs
Subject 2 has 12 runs
Subject 3 has 12 runs
Subject 4 has 12 runs
Subject 5 has 11 runs
Subject 6 has 12 runs
Running confound regression for res=2
Subject 1 has 12 runs
Subject 2 has 12 runs
Subject 3 has 12 runs
Subject 4 has 12 runs
Subject 5 has 11 runs
Subject 6 has 12 runs


### select task block timepoints

Drop the first two TRs from each task block to account for hemodynamic delay, and generate task labels for each timepoint.  First we extract the  necessary metadata:

In [6]:
# find the condition label for timepoints that are beyond the intial 4 seconds
def get_cond_info(
    layout, deriv_layout, t_r, cleaned_images, blocklen=20, n_trials_to_skip=2
):
    cond_info = {}

    subjects = [int(sub) for sub in layout.get_subjects()]
    for subject in subjects:
        runs = get_subject_runs(subject, data_dir)
        cond_info[subject] = {}
        for run in runs:
            events_file = layout.get(
                subject=subject,
                run=run,
                datatype='func',
                extension='tsv',
                return_type='file',
            )[0]
            events = pd.read_csv(events_file, sep='\t')
            n_timepoints = cleaned_images[subject][run][1].shape[-1]
            timepoints = np.arange(0, n_timepoints * t_r, t_r)

            # find task onsets in the events file
            # skip 2 trials i.e. 4 seconds
            blocklen = (
                20  # block length in seconds after removing first 4 seconds
            )

            conditions = events.trial_type.unique().tolist()
            conditions.sort()
            onsets = {}
            for condition in conditions:
                match_df = events[events.trial_type == condition]
                onsets[condition] = match_df.onset.tolist()[n_trials_to_skip]
            cond_df = pd.DataFrame(
                {'timepoint': timepoints, 'condition': None}
            )
            for idx in cond_df.index:
                for condition in conditions:
                    if cond_df.loc[idx, 'timepoint'] >= onsets[
                        condition
                    ] and cond_df.loc[idx, 'timepoint'] < (
                        onsets[condition] + blocklen
                    ):
                        cond_df.loc[idx, 'condition'] = condition
            for cond in cond_df.condition.unique():
                if cond is None:
                    continue
                assert len(cond_df[cond_df.condition == cond]) == 8
            cond_info[subject][run] = cond_df
    return cond_info


cond_info = get_cond_info(layout, deriv_layout, t_r, cleaned_images)

Then we extract the task images.

In [7]:


def get_task_images(cond_info, cleaned_images):
    task_images = {}
    task_info = {}
    for subject, runs in cond_info.items():
        task_images[subject] = {}
        task_info[subject] = {}
        for run, cond_df in runs.items():
            good_trials = cond_df.dropna()
            assert (
                len(good_trials) == 64
            ), f'Found {len(good_trials)} good trials for subject {subject} run {run}'
            task_img_file = cleaned_images[subject][run][0].replace(
                'cleaned', 'task'
            )
            assert task_img_file != cleaned_images[subject][run][0]
            good_trials.to_csv(
                task_img_file.replace('_bold.nii.gz', '_events.tsv'),
                sep='\t',
                index=False,
            )
            task_info[subject][run] = good_trials
            if not os.path.exists(task_img_file):
                cleaned_img = cleaned_images[subject][run][1]
                task_data = cleaned_img.get_fdata()[
                    :, :, :, list(good_trials.index)
                ]
                task_img = nib.Nifti1Image(task_data, cleaned_img.affine)
                task_img.to_filename(task_img_file)
            else:
                task_img = nib.load(task_img_file)
            task_images[subject][run] = task_img
    return task_images, task_info


task_images, task_info = get_task_images(cond_info, cleaned_images)

### Create aligned temporal cortex masks

We would like to reuse the masks shared by Haxby et al. for the visual cortex (which are included in the nilearn distribution).  This requires installation of the [ANTs software](https://stnava.github.io/ANTs/) and [FSL](https://fsl.fmrib.ox.ac.uk/fsl/docs/#/).

In [8]:
# First, we fetch single subject specific data with haxby datasets: to have
# anatomical image, EPI images and masks images

haxby_dataset = datasets.fetch_haxby(subjects=list(range(1, 7)))
haxby_datadir = Path(os.path.dirname(haxby_dataset.mask))

maskderivdir = Path(data_dir) / "derivatives/vtmasks"
if not maskderivdir.exists():
    maskderivdir.mkdir()

maskfiles = list(haxby_datadir.glob('sub*/mask4_vt.nii.gz'))
assert len(maskfiles) == 6

res = 3 # resampling resolution in mm

for maskfile in maskfiles:
    subject = int(maskfile.parts[-2].replace('subj',''))
    
    print(subject)
    maskimg = nib.load(str(maskfile))

    boldfiles = layout.get(suffix='bold', extension='nii.gz', run=1, subject=subject,
                            return_type='file')
    assert len(boldfiles) == 1
    boldimg = nib.load(boldfiles[0])

    maskimg_resampled = nib.Nifti1Image(maskimg.get_fdata(), boldimg.affine, boldimg.header)

    maskfile_resampled = maskderivdir / f'sub-{subject}_desc-mask4vt_resampled.nii.gz'
    maskimg_resampled.to_filename(str(maskfile_resampled))

    # call antsApplyTransform through the shell script
    print(run_shell_cmd(f'bash align_mask.sh {subject} {res}'))

    # load the aligned mask and check it
    mask_aligned = nib.load(str(maskderivdir / f'sub-{subject}_mask4vt_space-MNI152NLin2009cAsym_res-{res}.nii.gz'))
    mask_aligned_data = mask_aligned.get_fdata()
    print('max in realigned mask:', mask_aligned_data.max())

    


[get_dataset_dir] Dataset found in /Users/poldrack/nilearn_data/haxby2001
1
([], [])
max in realigned mask: 1.0
6
([], [])
max in realigned mask: 1.0
5
([], [])
max in realigned mask: 1.0
2
([], [])
max in realigned mask: 1.0
3
([], [])
max in realigned mask: 1.0
4
([], [])
max in realigned mask: 1.0


### save to HDF5 file

We will save all of the data to a single HDF5 file to make it easier to use them later.  

In [10]:
# This will stop the notebook from automatically running the rest of the cells
# which takes well over an hour
overwrite = False
assert overwrite, 'Set overwrite to True to run this cell'

AssertionError: Set overwrite to True to run this cell

In [ ]:
subjects = [int(sub) for sub in layout.get_subjects()]


difumo = datasets.fetch_atlas_difumo(
    dimension=1024, resolution_mm=3, legacy_format=False
)
difumo_masker = MultiNiftiMapsMasker(
    maps_img=difumo.maps,
    n_jobs=12,
)

vtmaskdir = os.path.join(data_dir, 'derivatives', 'vtmasks')
with h5py.File(os.path.join(output_dir, 'haxby_data_cleaned.h5'), 'w') as hf:

    for subject in subjects:
        print(f'Processing subject {subject}')
        g1 = hf.create_group(f'sub-{subject}')

        runs = get_subject_runs(subject, data_dir)
        sub_mask = get_subject_common_brain_mask(subject, data_dir)
        masker = NiftiMasker(mask_img=sub_mask)
        vt_mask = nib.load(
            os.path.join(
                vtmaskdir,
                f'sub-{subject}_mask4vt_space-MNI152NLin2009cAsym_res-3.nii.gz',
            )
        )

        vtmasker = NiftiMasker(mask_img=vt_mask)
        for run in runs:
            g2 = g1.create_group(f'run-{run}')

            vt_data = vtmasker.fit_transform(task_images[subject][run])
            assert vt_data.shape[0] == task_images[subject][run].shape[-1]
            assert vt_data.shape[1] == np.sum(vt_mask.get_fdata())
            g2.create_dataset('vtmaskdata', data=vt_data)

            # get the whole brain data
            braindata = masker.fit_transform(task_images[subject][run])
            assert braindata.shape[0] == task_images[subject][run].shape[-1]
            assert braindata.shape[1] == np.sum(sub_mask.get_fdata())
            g2.create_dataset('braindata', data=braindata)

            g2.create_dataset(
                'conditions',
                data=[
                    c.encode('utf-8')
                    for c in task_info[subject][run].condition.tolist()
                ],
            )

            # get the difumo data
            difumo_data = difumo_masker.fit_transform(
                task_images[subject][run]
            )
            assert difumo_data.shape[0] == task_images[subject][run].shape[-1]
            g2.create_dataset('difumodata', data=difumo_data)

[get_dataset_dir] Dataset found in /Users/poldrack/nilearn_data/difumo_atlases
Processing subject 1


/Users/poldrack/micromamba/envs/aineuro/lib/python3.12/site-packages/nilearn/maskers/nifti_masker.py:113: UserWarning: imgs are being resampled to the mask_img resolution. This process is memory intensive. You might want to provide a target_affine that is equal to the affine of the imgs or resample the mask beforehand to save memory and computation time.
  warnings.warn(
/Users/poldrack/micromamba/envs/aineuro/lib/python3.12/site-packages/nilearn/image/resampling.py:468: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0. 
Use 'force_resample=True' to suppress this warning.
  force_resample = _check_force_resample(force_resample)


[NiftiMasker.wrapped] Resampling images


/Users/poldrack/micromamba/envs/aineuro/lib/python3.12/site-packages/nilearn/maskers/nifti_masker.py:113: UserWarning: imgs are being resampled to the mask_img resolution. This process is memory intensive. You might want to provide a target_affine that is equal to the affine of the imgs or resample the mask beforehand to save memory and computation time.
  warnings.warn(
/Users/poldrack/micromamba/envs/aineuro/lib/python3.12/site-packages/nilearn/image/resampling.py:468: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0. 
Use 'force_resample=True' to suppress this warning.
  force_resample = _check_force_resample(force_resample)


[NiftiMasker.wrapped] Resampling images


...

[NiftiMasker.wrapped] Resampling images


/Users/poldrack/micromamba/envs/aineuro/lib/python3.12/site-packages/nilearn/maskers/nifti_masker.py:113: UserWarning: imgs are being resampled to the mask_img resolution. This process is memory intensive. You might want to provide a target_affine that is equal to the affine of the imgs or resample the mask beforehand to save memory and computation time.
  warnings.warn(
/Users/poldrack/micromamba/envs/aineuro/lib/python3.12/site-packages/nilearn/image/resampling.py:468: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0. 
Use 'force_resample=True' to suppress this warning.
  force_resample = _check_force_resample(force_resample)


[NiftiMasker.wrapped] Resampling images


/Users/poldrack/micromamba/envs/aineuro/lib/python3.12/site-packages/nilearn/maskers/nifti_masker.py:113: UserWarning: imgs are being resampled to the mask_img resolution. This process is memory intensive. You might want to provide a target_affine that is equal to the affine of the imgs or resample the mask beforehand to save memory and computation time.
  warnings.warn(
/Users/poldrack/micromamba/envs/aineuro/lib/python3.12/site-packages/nilearn/image/resampling.py:468: FutureWarning: 'force_resample' will be set to 'True' by default in Nilearn 0.13.0. 
Use 'force_resample=True' to suppress this warning.
  force_resample = _check_force_resample(force_resample)


[NiftiMasker.wrapped] Resampling images


### Average within blocks



In [ ]:
with h5py.File(os.path.join(output_dir, 'haxby_data_cleaned.h5'), 'a') as hf:

    subjects = [int(sub) for sub in layout.get_subjects()]
    for subject in subjects:
        runs = get_subject_runs(subject, data_dir)
        for run in runs:
            conditions = [
                i.decode('utf-8')
                for i in hf[f'sub-{subject}/run-{run}']['conditions'][:]
            ]

            vt_df = pd.DataFrame(
                hf[f'sub-{subject}/run-{run}']['vtmaskdata'][:]
            )
            condition_mean_df = vt_df.groupby(conditions).mean().sort_index()
            conditions_collapsed = condition_mean_df.index.values
            hf[f'sub-{subject}/run-{run}'].create_dataset(
                'mean_vtmaskdata', data=condition_mean_df
            )
            hf[f'sub-{subject}/run-{run}'].create_dataset(
                'mean_conditions', data=condition_mean_df.index.values
            )

            # same for whole brain
            brain_df = pd.DataFrame(
                hf[f'sub-{subject}/run-{run}']['braindata'][:]
            )
            condition_mean_df = (
                brain_df.groupby(conditions).mean().sort_index()
            )
            # make sure conditions are the same between the two
            assert np.all(
                condition_mean_df.index.values == conditions_collapsed
            )
            hf[f'sub-{subject}/run-{run}'].create_dataset(
                'mean_braindata', data=condition_mean_df
            )

            # same for difumo
            difumo_df = pd.DataFrame(
                hf[f'sub-{subject}/run-{run}']['difumodata'][:]
            )
            condition_mean_df = (
                difumo_df.groupby(conditions).mean().sort_index()
            )
            # make sure conditions are the same between the two
            assert np.all(
                condition_mean_df.index.values == conditions_collapsed
            )
            hf[f'sub-{subject}/run-{run}'].create_dataset(
                'mean_difumodata', data=condition_mean_df
            )

In [ ]:
condition_mean_df.sort_index()

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
bottle,20.341734,-22.073200,30.922354,81.861571,10.665755,29.259479,26.578807,37.862923,25.216348,82.032037,...,-14.311384,52.877827,60.923288,-41.514604,2.015644,-99.266953,18.120706,54.487036,50.416629,35.946949
cat,73.022439,71.218620,46.982960,-99.943771,-82.997792,-19.376414,-14.533978,14.493988,-52.933645,-23.939585,...,10.189615,-53.768042,-146.085152,-10.787128,20.850278,-4.142285,7.458493,-76.636407,-1.792390,-6.332361
chair,82.101613,48.521353,-88.396513,-89.974685,-0.374359,35.881146,-70.357718,29.602928,53.560321,-82.957480,...,-52.619997,-166.651067,37.305991,44.897594,34.454688,4.210086,-2.724119,33.191607,-55.610099,-53.686152
face,-28.963589,-70.509686,57.058891,40.271820,10.561323,70.224889,84.412823,-60.448015,39.220059,45.625593,...,-20.596774,147.662777,-64.983100,-61.369503,-93.389251,51.616918,4.501937,17.277040,77.824657,51.903456
house,2.912803,12.882667,-7.099153,-7.423538,25.423917,-45.242598,57.412470,11.389244,6.948350,14.118769,...,48.796809,32.558964,-21.311686,-31.977507,63.587849,-34.579945,-9.206479,-13.823469,-15.670533,25.304241
scissors,30.187111,3.495322,55.949960,-78.546075,-24.779875,-27.612369,82.711626,61.587111,-40.623987,0.200194,...,-61.802673,-4.264875,29.472675,-20.350482,-90.505631,44.947990,14.169799,-33.443818,-3.009493,66.552137
scrambledpix,42.561030,5.114049,6.168402,-67.019671,-83.211353,-24.716696,-11.574379,-59.429898,-39.038987,24.214875,...,-87.852859,42.776716,-105.855710,19.343371,-37.621537,37.560987,-0.718161,8.677190,117.592131,16.541259
shoe,-42.608003,37.353265,6.704696,-168.859276,5.502879,7.342246,-77.216618,-90.939512,-50.614532,8.119633,...,39.158401,-122.901523,29.642258,37.121569,-78.808787,21.077915,3.451898,-33.081502,-34.341228,-41.835821


In [ ]:

with h5py.File(os.path.join(output_dir, 'haxby_data_cleaned.h5'), 'r') as hf:

    # utils.list_all_datasets(hf)
    print(hf[f'sub-{subject}/run-{run}']['braindata'].shape)
    print(hf[f'sub-{subject}/run-{run}']['vtmaskdata'].shape)
    print(hf[f'sub-{subject}/run-{run}']['difumodata'].shape)

    print(hf[f'sub-{subject}/run-{run}']['conditions'].shape)
    print(hf[f'sub-{subject}/run-{run}']['mean_conditions'].shape)

(64, 71991)
(64, 746)
(64, 1024)
(64,)
(8,)
